In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,roc_auc_score,classification_report,recall_score,precision_score
from sklearn.svm import SVC
import numpy as np
from sklearn.model_selection import KFold


In [3]:
features_2d = pd.read_csv('./Data/2D_t2/all.csv')
data = features_2d.drop(columns=['Center'])

In [4]:

correlation = data.iloc[:,2:].corr(method='spearman')

In [5]:
selected_features = correlation[(correlation.abs() <= 0.6) | (correlation.abs() == 1)]

for i in range(200):
    for index in range(selected_features.shape[0]):
        row = selected_features.iloc[index]
        to_drop = row[row.isna()].index
        selected_features_c_droped = selected_features.drop(labels=to_drop,axis=0)
        selected_features_r_droped = selected_features_c_droped.drop(labels=to_drop,axis=1)
        if selected_features_r_droped.shape[0]<selected_features.shape[0]:
            break
    selected_features = selected_features_r_droped

In [8]:
selected_data = data[selected_features.index.to_list()+['Label','Name']]

In [9]:
selected_data

,median-Raw intensity,var-Raw intensity,kurtosis-Raw intensity,kurtosis-Gray std-dev ws=3,skewness-Gray mean ws=5,kurtosis-Gray range ws=5,median-Gradient sobelx,skewness-Gradient sobelx,median-Gradient sobely,skewness-Gradient sobely,...,Collage_kurt_InformationMeasureOfCorrelation2_1_nb_16_ws_7,Collage_kurt_MaximalCorrelationCoefficient_1_nb_16_ws_7,Collage_var_DifferenceVariance_1_nb_32_ws_7,Collage_skew_DifferenceVariance_1_nb_32_ws_7,Collage_var_InformationMeasureOfCorrelation2_1_nb_32_ws_7,Collage_kurt_SumAverage_1_nb_64_ws_7,Collage_skew_InformationMeasureOfCorrelation2_1_nb_64_ws_7,Collage_kurt_InformationMeasureOfCorrelation2_1_nb_64_ws_7,Label,Name
0,0.109351,0.003266,4.088218,3.662712,1.151848,2.611763,0.001063,-0.050111,0.005360,-0.078389,...,-0.496614,1.101392,0.764720,-0.110023,0.003181,-0.118047,0.032915,-0.544991,1,ahn_05
1,0.287552,0.024884,2.614300,2.934779,0.736331,2.073024,-0.046469,0.177711,0.039866,-0.409460,...,-0.141546,4.029511,0.942012,-0.051101,0.002383,-0.295207,-0.311675,-0.346096,1,ahn_09
2,0.230861,0.015638,2.111098,1.875319,0.663278,2.644442,0.058688,-0.176643,0.029497,0.060991,...,0.689030,0.623630,1.222423,-0.343912,0.001424,-0.814305,1.467952,0.695625,1,ahn_11
3,0.413685,0.017683,2.361455,4.574869,0.281675,2.491706,0.029273,0.454130,0.039714,-0.806455,...,0.101757,1.473815,0.407999,-0.248363,0.001681,-0.582250,0.517304,-0.504314,1,ahn_12
4,0.233332,0.006625,1.730104,1.961558,0.396965,2.548473,-0.006847,0.025814,0.003571,-0.090288,...,1.458600,-0.503830,0.322859,-0.093703,0.001102,-1.064062,0.588634,0.096634,1,ahn_16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,0.191394,0.006408,3.476188,3.386113,0.775637,2.786329,0.056471,0.004325,0.002672,0.552574,...,-0.760875,-0.928583,0.230905,0.245453,0.003536,-0.673987,-0.035935,-0.789425,1,nyu_0165
354,0.314138,0.027427,3.450835,5.169103,1.036193,3.577069,0.029763,-0.510894,0.010379,0.122131,...,-0.530060,2.724225,0.584243,-0.690390,0.003301,-0.559617,0.078450,-0.459258,1,nyu_0166
355,0.270596,0.007747,2.595020,5.849150,0.504539,3.510037,0.011048,-0.232108,-0.000849,-0.593981,...,-0.268240,6.022516,0.606795,-0.536334,0.003058,-0.587903,0.620985,0.571788,2,nyu_0168
356,0.600188,0.036941,1.829004,3.113218,-0.425899,1.858347,-0.005805,0.117475,-0.004003,-0.198477,...,0.292869,7.718691,0.385783,-0.764778,0.002382,0.210331,0.702142,0.896086,1,nyu_0172


In [11]:
def get_results(acc_list,auc_list,recall_list,precision_list):    
    acc_list = np.array(acc_list)
    auc_list = np.array(auc_list)
    recall_list = np.array(recall_list)
    precision_list = np.array(precision_list)
    print(f'Recall, Average:{recall_list.mean():.4f}, Std:{recall_list.std():.4f}')
    print(f'precision, Average:{precision_list.mean():.4f}, Std:{precision_list.std():.4f}')
    print(f'Accuracy, Average:{acc_list.mean():.4f}, Std:{acc_list.std():.4f}')
    print(f'AUC, Average:{auc_list.mean():.4f}, Std:{auc_list.std():.4f}')

In [174]:
val_acc_list = []
val_auc_list = []
val_recall_list = []
val_precision_list = []

test_acc_list = []
test_auc_list = []
test_recall_list = []
test_precision_list = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_index, test_index) in enumerate(kf.split(selected_data)):
    train_data = selected_data.iloc[train_index]
    val_data = selected_data.iloc[test_index]
    # print(train_data.shape)
    train_y = train_data[['Label']]
    train_x = train_data.iloc[:,:-1]
    
    val_y = val_data[['Label']]
    val_x = val_data.iloc[:,:-1]

    standard_scaler = StandardScaler()
    train_x = standard_scaler.fit_transform(train_x)
    val_x = standard_scaler.transform(val_x)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    # Train the model with validation data
    model.fit(train_x, train_y)
    
    y_pred_proba = model.predict(val_x)  # Get probabilities for the positive class
    y_pred = model.predict(val_x)

    accuracy = accuracy_score(val_y, y_pred)
    # print(f'Accuracy: {accuracy:.4f}')
    val_acc_list.append(accuracy)
    
    auc_score = roc_auc_score(val_y, y_pred_proba)
    # print(f'AUC Score: {auc_score:.4f}')
    val_auc_list.append(auc_score)
    
    # Print the classification report
    recall = recall_score(val_y, y_pred)
    # print(f'Recall Score: {recall:.4f}')
    val_recall_list.append(recall)
    
    precision = precision_score(val_y, y_pred)
    # print(f'AUC Score: {precision:.4f}')
    val_precision_list.append(precision)
    
print('Validation set')
get_results(val_acc_list,val_auc_list,val_recall_list,val_precision_list)
# print('--------------------')
# print("Test set")
# get_results(test_acc_list,test_auc_list,test_recall_list,test_precision_list)
print('================================================')

/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/anaconda3/envs/medical-image/lib/python3.10/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed

Validation set
Recall, Average:0.8391, Std:0.0398
precision, Average:0.7149, Std:0.0304
Accuracy, Average:0.7010, Std:0.0323
AUC, Average:0.6646, Std:0.0383


In [12]:
val_acc_list = []
val_auc_list = []
val_recall_list = []
val_precision_list = []

test_acc_list = []
test_auc_list = []
test_recall_list = []
test_precision_list = []
for fold in range(5):
        
    print(f"Taining Fold{fold+1} .....")
    test_file = file['test_split']
    train_file = file['cross_validation'][fold]['train_files']
    val_file = file['cross_validation'][fold]['validation_files']
    
    data = pd.read_csv(r'F:\Myproject\ISBI-Lung-project\Lung_data.csv')
    tarin_data = data[data['Image'].isin(train_file)]
    val_data = data[data['Image'].isin(val_file)]
    test_data = data[data['Image'].isin(test_file)]

    train_x = tarin_data.iloc[:,:-2]
    train_y = tarin_data[["label"]]

    val_x = val_data.iloc[:,:-2]
    val_y = val_data[["label"]]

    # test_x = test_data.iloc[:,:-2]
    # test_y = test_data[["label"]]
    
    standard_scaler = StandardScaler()
    train_x = standard_scaler.fit_transform(train_x)
    val_x = standard_scaler.transform(val_x)
    # test_x = standard_scaler.transform(test_x)

    model = RandomForestClassifier(n_estimators=100, random_state=42)
    # Train the model with validation data
    model.fit(train_x, train_y)
    
    
    
    # -------------------validation----------------
    y_pred_proba = model.predict(val_x)  # Get probabilities for the positive class
    y_pred = model.predict(val_x)

    accuracy = accuracy_score(val_y, y_pred)
    # print(f'Accuracy: {accuracy:.4f}')
    val_acc_list.append(accuracy)
    
    auc_score = roc_auc_score(val_y, y_pred_proba)
    # print(f'AUC Score: {auc_score:.4f}')
    val_auc_list.append(auc_score)
    
    # Print the classification report
    recall = recall_score(val_y, y_pred)
    # print(f'Recall Score: {recall:.4f}')
    val_recall_list.append(recall)
    
    precision = precision_score(val_y, y_pred)
    # print(f'AUC Score: {precision:.4f}')
    val_precision_list.append(precision)
    
    
    # Get predicted probabilities
    # -------------------testing--------------
    # y_pred_proba = model.predict(test_x)  # Get probabilities for the positive class
    # y_pred = model.predict(test_x)

    # accuracy = accuracy_score(test_y, y_pred)
    # # print(f'Accuracy: {accuracy:.4f}')
    # test_acc_list.append(accuracy)
    
    # auc_score = roc_auc_score(test_y, y_pred_proba)
    # # print(f'AUC Score: {auc_score:.4f}')
    # test_auc_list.append(auc_score)
    
    # # Print the classification report
    # recall = recall_score(test_y, y_pred)
    # # print(f'Recall Score: {recall:.4f}')
    # test_recall_list.append(recall)
    
    # precision = precision_score(test_y, y_pred)
    # # print(f'AUC Score: {precision:.4f}')
    # test_precision_list.append(precision)
    
print('Validation set')
get_results(val_acc_list,val_auc_list,val_recall_list,val_precision_list)
print('--------------------')
# print("Test set")
# get_results(test_acc_list,test_auc_list,test_recall_list,test_precision_list)
print('================================================')

    

Taining Fold1 .....


f:\Anaconda3\envs\radiomics\lib\site-packages\ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Taining Fold2 .....


f:\Anaconda3\envs\radiomics\lib\site-packages\ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Taining Fold3 .....


...

Taining Fold4 .....


f:\Anaconda3\envs\radiomics\lib\site-packages\ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Taining Fold5 .....


f:\Anaconda3\envs\radiomics\lib\site-packages\ipykernel_launcher.py:38: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Validation set
Recall, Average:0.5908, Std:0.0988
precision, Average:0.6568, Std:0.0757
Accuracy, Average:0.6646, Std:0.0444
AUC, Average:0.6617, Std:0.0441
--------------------
Test set
Recall, Average:0.7833, Std:0.0408
precision, Average:0.6450, Std:0.0380
Accuracy, Average:0.7706, Std:0.0288
AUC, Average:0.7735, Std:0.0293
